In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.pipeline import Pipeline


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Embedding,  Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

from tensorflow.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from tensorflow import keras
from sklearn import metrics
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

In [ ]:
import pandas as pd
import numpy as np
df_train=pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
df_train.head()

In [ ]:
df_test=pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
df_test.head()

In [ ]:
#n_missing column has the count of total missing values in each row
df_train['n_missing']=df_train.isna().sum(axis=1)
df_test['n_missing']=df_test.isna().sum(axis=1)

In [ ]:
df_train.head()

In [ ]:
#Changing the datatype of target column to Object.
df_train['claim']=df_train['claim'].astype(str)

In [ ]:
df_train['claim'].dtype

In [ ]:
#Storing all the column names in features,except claim and id.
features=[col for col in df_train.columns if col not in ['claim','id']]

In [ ]:
#Creating a pipeline to fill missing values using SimpleImputer Median strategy and transforming the input data to 
#uniform distribution for better training of the model.
pipe=Pipeline([('imputer',SimpleImputer(strategy='mean',missing_values=np.nan)),
               ('scaler',QuantileTransformer(n_quantiles=64,output_distribution='uniform')),
               ('bin',KBinsDiscretizer(n_bins=64,encode='ordinal',strategy='uniform'))])

In [ ]:
#Transforming the independant features
df_train[features]=pipe.fit_transform(df_train[features])

In [ ]:
df_test[features]=pipe.fit_transform(df_test[features])

In [ ]:
df_test[features].head()

In [ ]:
model = Sequential([
    Input(df_train[features].shape[1:]),
    Embedding(input_dim=64, output_dim=4),
    Flatten(),
    Dense(64,  activation='relu'),
    Dropout(0.5),
     Dense(32,  activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])

auc = tf.keras.metrics.AUC(name='aucroc')
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=5e-4,
        decay_steps = 900,
        decay_rate= 0.9)
callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

optimizer = RMSprop(lr=5e-4, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=[auc]) 


In [ ]:
model.fit(x = np.float32(df_train[features]), y = np.float32(df_train.claim),
          batch_size = 1024, shuffle = True, epochs = 100,callbacks=[callback])

In [ ]:
sub=pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sub.head()

In [ ]:
sub['claim']=model.predict(np.float32(df_test[features]))

In [ ]:
sub=sub.set_index('id')

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
ls

In [ ]:
sub.head()